In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier
import category_encoders as ce


# import tensorflow as tf
# from tensorflow.keras import layers
# from tensorflow.keras import optimizers
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras import callbacks
# from tensorflow.keras import backend as K

In [ ]:
#Importing
train_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/training_v2.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/unlabeled.csv')

sub_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/samplesubmission.csv')
data_dict = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [ ]:
train_data.head()

In [ ]:
data_dict['Data Type'].unique()

In [ ]:
cat_var = list(data_dict[data_dict['Data Type'].isin(['binary','string'])]['Variable Name'])
cat_var.remove('icu_admit_type')
#cat_var.remove('bmi')
cat_var = cat_var[1:]

In [ ]:
#cat_var = cat_var[1:]

In [ ]:
cont_var = list(data_dict[data_dict['Data Type'].isin(['integer','numeric'])]['Variable Name'])
cont_var.remove('pred')
#cont_var = cont_var + ['bmi']

In [ ]:
test_data.isna().sum()

#### Baseline

In [ ]:
x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [ ]:
x_test = test_data.drop('hospital_death',axis=1)

In [ ]:
x_comb = pd.concat([x_train,x_test],sort=False)

In [ ]:
# # Remove Features with more than 70 percent missing values
# data_missing = (x_comb.isnull().sum() / len(x_comb)).sort_values(ascending = False)
# data_missing = data_missing.index[data_missing > 0.60]

# x_comb = x_comb.drop(columns = data_missing)
x_comb.isna().sum()

In [ ]:
#Imputing Missing values

In [ ]:
x_comb["apache_4a_hospital_death_prob"] = x_comb["apache_4a_hospital_death_prob"].replace(-1, np.nan)
x_comb["apache_4a_icu_death_prob"] = x_comb["apache_4a_icu_death_prob"].replace(-1, np.nan)

In [ ]:
new_cat_var = list(set(cat_var).intersection(set(x_comb.columns)))

In [ ]:
for col in tqdm(new_cat_var):
    #x_comb[col] = x_comb[col].fillna(x_comb[col].mode().values[0])
    x_comb[col] = x_comb[col].fillna('unk')

In [ ]:
new_cont_var = list(set(cont_var).intersection(set(x_comb.columns)))

In [ ]:
for col in tqdm(new_cont_var):
    x_comb[col] = x_comb[col].fillna(x_comb[col].mean())

In [ ]:
for col in tqdm(new_cat_var):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

In [ ]:
# for i,col in tqdm(enumerate(cont_var)):
#     #mu = x_comb[col].mean()
#     #std = x_comb[col].std()
#     #x_comb[col] = x_comb[col].apply(lambda x:(x-mu)/(std))
#     x_comb[col] = np.log1p(x_comb[col])


In [ ]:
for i,col in tqdm(enumerate(new_cont_var)):
    counter = Counter(x_comb[col])
    x_comb[str(col)+'count'] = x_comb[col].apply(lambda x:counter[x])

##### Target Encoding

In [ ]:
temp_train = x_train.copy()
temp_train['hospital_death'] = train_data['hospital_death']

In [ ]:
cat_count_var = []

for i,col in tqdm(enumerate(cat_var)):
    mean = temp_train['hospital_death'].mean()
    
    #Compute the number of values and the mean of each group
    agg = temp_train.groupby(col)['hospital_death'].agg(['count', 'mean'])
    counts = agg['count']
    means =  agg['mean']
    
    #Compute the "smoothed" means
    m=3
    smooth = (counts * means + m * mean) / (counts + m)
    
    #Final_val
    x_comb[str(col)+'count_new'] = x_comb[col].map(smooth)
    cat_count_var.append(str(col)+'count_new')
    

In [ ]:
# # Remove Features with more than 70 percent missing values
data_missing = (x_comb.isnull().sum() / len(x_comb)).sort_values(ascending = False)
data_missing = data_missing.index[data_missing > 0.90]

x_comb = x_comb.drop(columns = data_missing)

In [ ]:
cat_count_var_new = list(set(cat_count_var).intersection(x_comb.columns))

In [ ]:
for col in tqdm(cat_count_var_new):
    le = LabelEncoder()
    x_comb[col] = le.fit_transform(x_comb[col].astype(str))

##### Aggregate Features

In [ ]:
#cat_count_var

In [ ]:
aggs = {}

for col in tqdm(new_cat_var):
    aggs[col] = ['count', 'nunique']
    

In [ ]:
for col in tqdm(new_cat_var):
    
    aggs_temp = aggs.copy()
    aggs_temp.pop(col)
    
    agg_df = x_comb.groupby(col).agg(aggs_temp).reset_index()
    agg_df.columns = [col] + [col + '_' + c[0] +'_' + c[1] for c in agg_df.columns[1:]]
    
    x_comb = pd.merge(left = x_comb, right= agg_df, how='left',
                    left_on=[col], right_on=[col])
    

In [ ]:
x_comb = x_comb.drop(['encounter_id','patient_id'],axis=1)

In [ ]:
x_comb.shape

In [ ]:
train_x = x_comb[0:91713]
test_x = x_comb[91713:]

In [ ]:
x1_train, x1_val, y1_train, y1_val = train_test_split(train_x, y_train, 
                                                      test_size=0.2,
                                                      random_state=42,
                                                      stratify = y_train)

In [ ]:
# from imblearn.over_sampling import SMOTE 
# sm = SMOTE(random_state = 2) 

In [ ]:
#x_train_res, y_train_res = sm.fit_sample(x1_train, y1_train)

In [ ]:
#let us make the catboost model, use_best_model params will make the model prevent overfitting
model = CatBoostClassifier(iterations=3000, 
                           learning_rate=0.04, 
                           l2_leaf_reg= 3.5,  
                           depth= 6, 
                           rsm=0.98, 
                           loss_function= 'Logloss', 
                           eval_metric='AUC',
                           use_best_model=True,
                           random_seed=42)

In [ ]:
model.fit(x1_train,y1_train,cat_features= new_cat_var , eval_set=(x1_val,y1_val))


In [ ]:
#last let us make the submission,note that you have to make the pred to be int!
pred = model.predict_proba(test_x)
preds= pred[:,1]

In [ ]:
sub_data

In [ ]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = preds

In [ ]:
x_test.head()

In [ ]:
ls

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/sub7_catb_agg.csv',index = False)

In [ ]:
data_sub.shape